In [1]:
import pandas as pd

In [2]:
%matplotlib inline

## Read data and separate Switzerland, Cantons, Districts, Communes

### Load the Excel with all the population data in it

In [3]:
df = pd.read_excel('CleanData/SchweizerBev-Wide-1981-2015.xls')

In [4]:
df.head(2)

,Einheit,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Schweiz,5421746.0,5442311.0,5462782.0,5481246.0,5502490.0,5524160.0,5546455.0,5567111.0,5587259.0,...,5917216.0,5954212.0,5991401.0,6032141.0,6071802.0,6103857.0,6138668.0,6169091.0,6202184.0,6239207.0
1,- Zürich,938303.0,937908.0,938859.0,937551.0,939975.0,941702.0,943295.0,944201.0,944287.0,...,987385.0,998574.0,1007725.0,1019182.0,1031109.0,1042311.0,1050188.0,1057472.0,1064112.0,1073065.0


In [5]:
#Make sure we don't have any NaN in there
df = df[df['Einheit'].notnull()]

### Load some helper files

In [6]:
#Load the numbers and abbreviations file for the cantons
df_kant_ids = pd.read_csv('OriginalData/Kantone-ABK-ID.csv')
df_kant_ids.head(2)

,Kanton_ABK,Kanton_ID,Kanton_Name,Kanton_Name2
0,ZH,1,Zürich,Zürich
1,BE,2,Bern,Bern / Berne


In [7]:
#Load the canton/district infos for the communes
df_gem_ids = pd.read_excel('OriginalData/Gemeindestand-2015.xls')
df_gem_ids.head(2)

,Hist.-Nummer,Kanton,Bezirksnummer,Bezirksname,BFS Gde-nummer,Gemeindename,Datum der Aufnahme
0,13256,ZH,101,Affoltern,1,Aeugst am Albis,1976-11-15
1,11742,ZH,101,Affoltern,2,Affoltern am Albis,1960-01-01


In [8]:
df_gem_ids.pop('Hist.-Nummer')
df_gem_ids.pop('Gemeindename')
df_gem_ids.pop('Datum der Aufnahme')
df_gem_ids.rename(columns={'Bezirksnummer': 'Bezirk_ID', 'Bezirksname': 'Bezirk_Name', 'BFS Gde-nummer': 'Gemeinde_ID', 'Kanton': 'Kanton_ABK'}, inplace=True)
df_gem_ids.head(2)

,Kanton_ABK,Bezirk_ID,Bezirk_Name,Gemeinde_ID
0,ZH,101,Affoltern,1
1,ZH,101,Affoltern,2


In [9]:
#Construct a set of district ids
df_bez_ids = df_gem_ids.drop_duplicates(subset='Bezirk_ID')
df_bez_ids.pop('Gemeinde_ID')
df_bez_ids.head(2)

,Kanton_ABK,Bezirk_ID,Bezirk_Name
0,ZH,101,Affoltern
14,ZH,102,Andelfingen


## Get a dataframe for each geopraphic level

### Separate Switzerland

In [10]:
#Switzerland
df_ch = df[df['Einheit'] == 'Schweiz']
df_ch.pop('Einheit')
df_ch.head(2)

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,5421746.0,5442311.0,5462782.0,5481246.0,5502490.0,5524160.0,5546455.0,5567111.0,5587259.0,5607711.0,...,5917216.0,5954212.0,5991401.0,6032141.0,6071802.0,6103857.0,6138668.0,6169091.0,6202184.0,6239207.0


In [11]:
df_ch.shape

(1, 35)

### Separate the cantons

In [12]:
#Cantons - there should be 26 of them
df_kant = df[df['Einheit'].str.contains('^- ')]
df_kant['Kanton_Name'] = df_kant['Einheit'].str.extract('^- (.*)', expand=True)
df_kant.head(2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Einheit,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,Kanton_Name
1,- Zürich,938303.0,937908.0,938859.0,937551.0,939975.0,941702.0,943295.0,944201.0,944287.0,...,998574.0,1007725.0,1019182.0,1031109.0,1042311.0,1050188.0,1057472.0,1064112.0,1073065.0,Zürich
183,- Bern / Berne,830246.0,832510.0,834861.0,836579.0,837728.0,838638.0,840026.0,841168.0,842393.0,...,841188.0,843052.0,845650.0,847153.0,848100.0,850232.0,853602.0,855740.0,857915.0,Bern / Berne


In [13]:
#Add the ID and Abbreviations
df_kant = df_kant.merge(df_kant_ids, how='inner', left_on='Kanton_Name', right_on='Kanton_Name2')
df_kant.pop('Kanton_Name2')
df_kant.pop('Einheit')
df_kant.pop('Kanton_Name_y')
df_kant.rename(columns={'Kanton_Name_x': 'Kanton_Name'}, inplace=True)
df_kant.head(2)

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2009,2010,2011,2012,2013,2014,2015,Kanton_Name,Kanton_ABK,Kanton_ID
0,938303.0,937908.0,938859.0,937551.0,939975.0,941702.0,943295.0,944201.0,944287.0,942558.0,...,1019182.0,1031109.0,1042311.0,1050188.0,1057472.0,1064112.0,1073065.0,Zürich,ZH,1
1,830246.0,832510.0,834861.0,836579.0,837728.0,838638.0,840026.0,841168.0,842393.0,843916.0,...,845650.0,847153.0,848100.0,850232.0,853602.0,855740.0,857915.0,Bern / Berne,BE,2


In [14]:
df_kant.shape

(26, 38)

### Separate the communes

In [15]:
#Gemeinden - there should be 2324 of them
df_gem = df[df['Einheit'].str.contains('^\.\.\.\.\.\.')]
df_gem['Gemeinde_ID'] = df_gem['Einheit'].str.extract('^\.\.\.\.\.\.(\d\d\d\d)', expand=True).astype(int)
df_gem['Gemeinde_Name'] = df_gem['Einheit'].str.extract('^\.\.\.\.\.\.\d\d\d\d (.*)', expand=True)
df_gem.head(2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Einheit,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2008,2009,2010,2011,2012,2013,2014,2015,Gemeinde_ID,Gemeinde_Name
3,......0001 Aeugst am Albis,834.0,875.0,910.0,925.0,929.0,962.0,967.0,1003.0,1055.0,...,1538.0,1536.0,1538.0,1599.0,1651.0,1678.0,1689.0,1699.0,1,Aeugst am Albis
4,......0002 Affoltern am Albis,6597.0,6706.0,6784.0,6925.0,7119.0,7189.0,7286.0,7361.0,7462.0,...,7760.0,7954.0,8022.0,8259.0,8260.0,8340.0,8423.0,8463.0,2,Affoltern am Albis


In [16]:
#Add the district and canton IDs and names
df_gem = df_gem.merge(df_gem_ids, how='inner', on='Gemeinde_ID')
df_gem.pop('Einheit')
df_gem.head(2)

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2011,2012,2013,2014,2015,Gemeinde_ID,Gemeinde_Name,Kanton_ABK,Bezirk_ID,Bezirk_Name
0,834.0,875.0,910.0,925.0,929.0,962.0,967.0,1003.0,1055.0,1080.0,...,1599.0,1651.0,1678.0,1689.0,1699.0,1,Aeugst am Albis,ZH,101,Affoltern
1,6597.0,6706.0,6784.0,6925.0,7119.0,7189.0,7286.0,7361.0,7462.0,7602.0,...,8259.0,8260.0,8340.0,8423.0,8463.0,2,Affoltern am Albis,ZH,101,Affoltern


In [17]:
df_gem = df_gem.merge(df_kant_ids, how='inner', on='Kanton_ABK')
df_gem.pop('Kanton_Name2')
df_gem.head(2)

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2013,2014,2015,Gemeinde_ID,Gemeinde_Name,Kanton_ABK,Bezirk_ID,Bezirk_Name,Kanton_ID,Kanton_Name
0,834.0,875.0,910.0,925.0,929.0,962.0,967.0,1003.0,1055.0,1080.0,...,1678.0,1689.0,1699.0,1,Aeugst am Albis,ZH,101,Affoltern,1,Zürich
1,6597.0,6706.0,6784.0,6925.0,7119.0,7189.0,7286.0,7361.0,7462.0,7602.0,...,8340.0,8423.0,8463.0,2,Affoltern am Albis,ZH,101,Affoltern,1,Zürich


In [18]:
df_gem.shape

(2324, 42)

### Separate the districts

In [19]:
#We have to do this one differently
df_bez = df_gem.groupby('Bezirk_ID').sum().reset_index()
df_bez.pop('Gemeinde_ID')
df_bez.pop('Kanton_ID')
df_bez.head(2)

,Bezirk_ID,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,101,25989.0,26703.0,27299.0,28123.0,28751.0,29065.0,29457.0,29940.0,30427.0,...,37602.0,38222.0,38631.0,39176.0,39729.0,40432.0,40914.0,41168.0,41428.0,41720.0
1,102,19694.0,19795.0,19852.0,20026.0,20287.0,20455.0,20646.0,20906.0,21189.0,...,25294.0,25485.0,25675.0,25894.0,26189.0,26210.0,26398.0,26521.0,26581.0,26749.0


In [20]:
df_bez = df_bez.merge(df_bez_ids, how='inner', on='Bezirk_ID')
df_bez.head(2)

,Bezirk_ID,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2008,2009,2010,2011,2012,2013,2014,2015,Kanton_ABK,Bezirk_Name
0,101,25989.0,26703.0,27299.0,28123.0,28751.0,29065.0,29457.0,29940.0,30427.0,...,38631.0,39176.0,39729.0,40432.0,40914.0,41168.0,41428.0,41720.0,ZH,Affoltern
1,102,19694.0,19795.0,19852.0,20026.0,20287.0,20455.0,20646.0,20906.0,21189.0,...,25675.0,25894.0,26189.0,26210.0,26398.0,26521.0,26581.0,26749.0,ZH,Andelfingen


In [21]:
df_bez = df_bez.merge(df_kant_ids, how='inner', on='Kanton_ABK')
df_bez.pop('Kanton_Name2')
df_bez.head(2)

,Bezirk_ID,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2010,2011,2012,2013,2014,2015,Kanton_ABK,Bezirk_Name,Kanton_ID,Kanton_Name
0,101,25989.0,26703.0,27299.0,28123.0,28751.0,29065.0,29457.0,29940.0,30427.0,...,39729.0,40432.0,40914.0,41168.0,41428.0,41720.0,ZH,Affoltern,1,Zürich
1,102,19694.0,19795.0,19852.0,20026.0,20287.0,20455.0,20646.0,20906.0,21189.0,...,26189.0,26210.0,26398.0,26521.0,26581.0,26749.0,ZH,Andelfingen,1,Zürich


## Export the data

In [22]:
#Switzerland
df_ch.to_csv('CleanerData/CH-SchweizerBev-Wide-1981-2015.csv', index=False)
df_temp = pd.read_csv('CleanerData/CH-SchweizerBev-Wide-1981-2015.csv')
df_temp.head(2)

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,5421746.0,5442311.0,5462782.0,5481246.0,5502490.0,5524160.0,5546455.0,5567111.0,5587259.0,5607711.0,...,5917216.0,5954212.0,5991401.0,6032141.0,6071802.0,6103857.0,6138668.0,6169091.0,6202184.0,6239207.0


In [23]:
#Cantons
df_kant.to_csv('CleanerData/Kant-SchweizerBev-Wide-1981-2015.csv', index=False)
df_temp = pd.read_csv('CleanerData/Kant-SchweizerBev-Wide-1981-2015.csv')
df_temp.head(2)

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2009,2010,2011,2012,2013,2014,2015,Kanton_Name,Kanton_ABK,Kanton_ID
0,938303.0,937908.0,938859.0,937551.0,939975.0,941702.0,943295.0,944201.0,944287.0,942558.0,...,1019182.0,1031109.0,1042311.0,1050188.0,1057472.0,1064112.0,1073065.0,Zürich,ZH,1
1,830246.0,832510.0,834861.0,836579.0,837728.0,838638.0,840026.0,841168.0,842393.0,843916.0,...,845650.0,847153.0,848100.0,850232.0,853602.0,855740.0,857915.0,Bern / Berne,BE,2


In [24]:
#Districts
df_bez.to_csv('CleanerData/Bez-SchweizerBev-Wide-1981-2015.csv', index=False)
df_temp = pd.read_csv('CleanerData/Bez-SchweizerBev-Wide-1981-2015.csv')
df_temp.head(2)

,Bezirk_ID,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2010,2011,2012,2013,2014,2015,Kanton_ABK,Bezirk_Name,Kanton_ID,Kanton_Name
0,101,25989.0,26703.0,27299.0,28123.0,28751.0,29065.0,29457.0,29940.0,30427.0,...,39729.0,40432.0,40914.0,41168.0,41428.0,41720.0,ZH,Affoltern,1,Zürich
1,102,19694.0,19795.0,19852.0,20026.0,20287.0,20455.0,20646.0,20906.0,21189.0,...,26189.0,26210.0,26398.0,26521.0,26581.0,26749.0,ZH,Andelfingen,1,Zürich


In [25]:
#Communes
df_gem.to_csv('CleanerData/Gem-SchweizerBev-Wide-1981-2015.csv', index=False)
df_temp = pd.read_csv('CleanerData/Gem-SchweizerBev-Wide-1981-2015.csv')
df_temp.head(2)

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2013,2014,2015,Gemeinde_ID,Gemeinde_Name,Kanton_ABK,Bezirk_ID,Bezirk_Name,Kanton_ID,Kanton_Name
0,834.0,875.0,910.0,925.0,929.0,962.0,967.0,1003.0,1055.0,1080.0,...,1678.0,1689.0,1699.0,1,Aeugst am Albis,ZH,101,Affoltern,1,Zürich
1,6597.0,6706.0,6784.0,6925.0,7119.0,7189.0,7286.0,7361.0,7462.0,7602.0,...,8340.0,8423.0,8463.0,2,Affoltern am Albis,ZH,101,Affoltern,1,Zürich
